# Assignment 01 — Email & Document Writer with Langfuse

## 🎯 프로젝트 개요
**주제**: 이메일·문서 자동 작성  
**목적**: Prompt 버전 관리 → 배포 → Tracing → Evaluation 전체 사이클 구현

### 평가 지표
1. **형식 준수율** (Schema Compliance): 필수 구조 요소 포함 여부
2. **필수 항목 포함률** (Mandatory Items): must_include 리스트 완성도
3. **톤 일치도** (Tone Consistency): LLM-as-Judge로 0-5점 평가
4. **길이 제한 준수** (Length Control): max_length 이내 작성 여부
5. **편집 필요도** (Edit Need): LLM-as-Judge로 0-5점 평가 (낮을수록 좋음)

### 버전 비교
- **V1.0.0 (dev)**: 기본 프롬프트 - 간단한 입력만 받음
- **V2.0.0 (production)**: 개선 프롬프트 - 상세 제약조건, 필수 요소 검증

## 📦 환경 설정

In [ ]:
!pip install -q langfuse openai python-dotenv

In [9]:
import os
import json
import time
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any

from dotenv import load_dotenv
from langfuse import Langfuse
from openai import OpenAI

# 환경 변수 로드
load_dotenv()

# API 키 설정
LANGFUSE_PUBLIC_KEY = os.getenv("LANGFUSE_PUBLIC_KEY")
LANGFUSE_SECRET_KEY = os.getenv("LANGFUSE_SECRET_KEY")
LANGFUSE_HOST = os.getenv("LANGFUSE_HOST", "https://cloud.langfuse.com")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

print("✅ Environment Check:")
print(f"  Langfuse Host: {LANGFUSE_HOST}")
print(f"  Langfuse Public Key: {'✓' if LANGFUSE_PUBLIC_KEY else '✗'}")
print(f"  Langfuse Secret Key: {'✓' if LANGFUSE_SECRET_KEY else '✗'}")
print(f"  OpenAI API Key: {'✓' if OPENAI_API_KEY else '✗'}")

# Langfuse 초기화
langfuse = Langfuse(
    public_key=LANGFUSE_PUBLIC_KEY,
    secret_key=LANGFUSE_SECRET_KEY,
    host=LANGFUSE_HOST
)

# OpenAI 클라이언트 초기화 (Langfuse wrapper 사용)
from langfuse.openai import openai
openai.api_key = OPENAI_API_KEY

print("\n✅ Langfuse 연결 성공!")

✅ Environment Check:
  Langfuse Host: https://cloud.langfuse.com
  Langfuse Public Key: ✓
  Langfuse Secret Key: ✓
  OpenAI API Key: ✓

✅ Langfuse 연결 성공!

✅ Langfuse 연결 성공!


## 📊 Dataset 로드 및 확인

In [2]:
# JSONL 데이터셋 로드
dataset_path = Path("datasets/email_writer_eval.jsonl")

def load_dataset(path: Path) -> List[Dict]:
    """JSONL 파일 로드"""
    data = []
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.strip()))
    return data

dataset = load_dataset(dataset_path)
print(f"✅ 데이터셋 로드 완료: {len(dataset)}개 샘플")
print(f"\n샘플 #1:")
print(json.dumps(dataset[0], indent=2, ensure_ascii=False))

✅ 데이터셋 로드 완료: 15개 샘플

샘플 #1:
{
  "id": "e001",
  "input": {
    "purpose": "출장 보고서",
    "audience": "임원진",
    "key_points": [
      "예산 집행 현황",
      "주요 미팅 결과",
      "리스크 요인"
    ]
  },
  "reference": {
    "style": "formal",
    "must_include": [
      "예산 수치",
      "리스크 2개 이상",
      "향후 조치사항"
    ],
    "tone": "professional",
    "max_length": 500
  }
}


## 🔧 Prompt 버전 로드 (V1 & V2)

In [3]:
# .prompty 파일 파싱 함수
def parse_prompty_file(path: Path) -> Dict[str, Any]:
    """Parse .prompty file and extract metadata + prompt"""
    with open(path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # Split frontmatter and prompt content
    parts = content.split('---')
    if len(parts) >= 3:
        # frontmatter = parts[1]  # YAML metadata (not parsing for now)
        prompt_content = parts[2].strip()
    else:
        prompt_content = content
    
    # Extract system and user sections
    system_prompt = ""
    user_prompt = ""
    
    if "system:" in prompt_content:
        parts = prompt_content.split("user:", 1)
        system_prompt = parts[0].replace("system:", "").strip()
        if len(parts) > 1:
            user_prompt = parts[1].strip()
    else:
        user_prompt = prompt_content.strip()
    
    return {
        "system": system_prompt,
        "user_template": user_prompt
    }

# V1, V2 프롬프트 로드
prompt_v1 = parse_prompty_file(Path("prompts/email_writer_v1.0.prompty"))
prompt_v2 = parse_prompty_file(Path("prompts/email_writer_v2.0.prompty"))

print("✅ V1 Prompt 로드 완료")
print(f"  System: {len(prompt_v1['system'])} chars")
print(f"  User Template: {len(prompt_v1['user_template'])} chars")
print("\n✅ V2 Prompt 로드 완료")
print(f"  System: {len(prompt_v2['system'])} chars")
print(f"  User Template: {len(prompt_v2['user_template'])} chars")

✅ V1 Prompt 로드 완료
  System: 132 chars
  User Template: 204 chars

✅ V2 Prompt 로드 완료
  System: 873 chars
  User Template: 647 chars


## 🚀 Langfuse에 Prompt 배포 (Versioning)

**중요**: Langfuse UI에서 수동으로 Prompt를 생성한 후, 여기서 fetch하여 사용합니다.  
또는 API로 직접 생성할 수도 있습니다.

In [4]:
# Langfuse에 Prompt 생성/업데이트
# 참고: Langfuse Python SDK에서는 create_prompt() 메서드 사용

PROMPT_NAME = "email-writer"

# V1 배포 (dev 라벨)
try:
    langfuse.create_prompt(
        name=PROMPT_NAME,
        prompt=prompt_v1['user_template'],
        config={
            "model": "gpt-4o-mini",
            "temperature": 0.7,
            "max_tokens": 800,
            "system_message": prompt_v1['system']
        },
        labels=["dev"],
        tags=["v1.0.0", "basic", "email-writing"]
    )
    print("✅ V1 Prompt 배포 완료 (dev label)")
except Exception as e:
    print(f"⚠️  V1 Prompt 이미 존재하거나 오류: {e}")

# V2 배포 (production 라벨)
try:
    langfuse.create_prompt(
        name=PROMPT_NAME,
        prompt=prompt_v2['user_template'],
        config={
            "model": "gpt-4o-mini",
            "temperature": 0.7,
            "max_tokens": 1000,
            "system_message": prompt_v2['system']
        },
        labels=["production"],
        tags=["v2.0.0", "professional", "email-writing", "structured"]
    )
    print("✅ V2 Prompt 배포 완료 (production label)")
except Exception as e:
    print(f"⚠️  V2 Prompt 이미 존재하거나 오류: {e}")

# Flush to ensure prompts are created
langfuse.flush()
time.sleep(2)
print("\n🔄 Langfuse에 Prompt 동기화 완료")

✅ V1 Prompt 배포 완료 (dev label)
✅ V2 Prompt 배포 완료 (production label)
✅ V2 Prompt 배포 완료 (production label)

🔄 Langfuse에 Prompt 동기화 완료

🔄 Langfuse에 Prompt 동기화 완료


## 📝 Email 생성 함수 (with Tracing)

Summarized conversation history선택하신 코드는 **Langfuse에 Prompt를 프로그래밍 방식으로 배포(생성)하는 코드**입니다. 각 부분을 설명드리겠습니다:

## 🎯 코드의 목적



In [ ]:
# langfuse.create_prompt(
#     name="email-writer",
#     prompt=prompt_v1['user_template'],
#     config={...},
#     labels=["dev"],
#     tags=["v1.0.0", "basic", "email-writing"]
# )



이 코드는 **Langfuse 플랫폼에 프롬프트를 등록**하여:
1. **버전 관리**: 프롬프트를 중앙에서 관리하고 버전을 추적
2. **재사용성**: 여러 곳에서 같은 프롬프트를 fetch하여 사용
3. **라벨링**: `dev`와 `production` 환경 구분
4. **추적**: 어떤 프롬프트 버전이 어떤 결과를 냈는지 추적 가능

---

## 📋 파라미터 설명

### 1. `name="email-writer"`
- **프롬프트 식별자**: Langfuse에서 이 프롬프트를 찾을 때 사용하는 이름
- 나중에 `langfuse.get_prompt("email-writer", label="dev")`로 불러올 수 있음

### 2. `prompt=prompt_v1['user_template']`
- **실제 프롬프트 텍스트**: `.prompty` 파일에서 파싱한 사용자 메시지 템플릿
- Jinja2 형식의 템플릿 문자열 (예: `{{purpose}}`, `{{key_points}}`)

### 3. `config={...}`
- **모델 설정**: OpenAI 호출 시 사용할 파라미터들
  ```python
  {
      "model": "gpt-4o-mini",        # 사용할 모델
      "temperature": 0.7,             # 창의성 정도 (0-1)
      "max_tokens": 800,              # 최대 출력 길이
      "system_message": prompt_v1['system']  # 시스템 프롬프트
  }
  ```

### 4. `labels=["dev"]` (V1) / `["production"]` (V2)
- **환경 라벨**: 개발/운영 환경 구분
  - `dev`: 테스트 중인 버전 (V1)
  - `production`: 실제 사용 중인 버전 (V2)
- 같은 이름의 프롬프트를 여러 버전으로 관리 가능

### 5. `tags=["v1.0.0", "basic", "email-writing"]`
- **메타데이터 태그**: 검색, 분류, 필터링 용도
  - 버전 번호: `v1.0.0`, `v2.0.0`
  - 특징: `basic`, `professional`, `structured`
  - 카테고리: `email-writing`

---

## 🔄 V1 vs V2 비교



In [ ]:
# # V1 (개발 버전)
# langfuse.create_prompt(
#     name="email-writer",
#     labels=["dev"],           # 개발 환경
#     tags=["v1.0.0", "basic"], # 기본 버전
#     config={"max_tokens": 800}
# )

# # V2 (프로덕션 버전)
# langfuse.create_prompt(
#     name="email-writer",
#     labels=["production"],          # 운영 환경
#     tags=["v2.0.0", "professional"], # 개선 버전
#     config={"max_tokens": 1000}     # 더 긴 출력 허용
# )



두 버전 모두 **같은 이름** (`email-writer`)이지만, **다른 라벨**로 구분됩니다.

---

## 💡 실제 사용 예시

1. **프롬프트 등록** (지금 실행하는 셀)
   ```python
   langfuse.create_prompt(name="email-writer", labels=["dev"], ...)
   ```

2. **나중에 불러오기**
   ```python
   # 개발 버전 사용
   prompt = langfuse.get_prompt("email-writer", label="dev")
   
   # 운영 버전 사용
   prompt = langfuse.get_prompt("email-writer", label="production")
   ```

3. **Langfuse UI에서 확인**
   - 웹 대시보드에서 프롬프트 히스토리 조회
   - 어떤 버전이 언제 배포되었는지 추적
   - A/B 테스트 결과 비교

---

## ⚠️ 주의사항



In [ ]:
# except Exception as e:
#     print(f"⚠️  V1 Prompt 이미 존재하거나 오류: {e}")



- **중복 방지**: 같은 이름+라벨로 이미 프롬프트가 있으면 오류 발생
- 재실행 시 오류 무시하고 계속 진행

---

**요약**: 이 코드는 `.prompty` 파일의 내용을 Langfuse 클라우드에 업로드하여 **중앙 집중식 프롬프트 버전 관리**를 시작하는 코드입니다. 마치 Git에 코드를 push하는 것처럼, 프롬프트를 Langfuse에 "배포"하는 것이죠! 🚀

In [16]:
def generate_email_v1(
    purpose: str,
    audience: str,
    key_points: List[str],
    trace_name: str = "email-generation-v1"
) -> str:
    """V1 프롬프트로 이메일 생성 (Langfuse Tracing)"""
    
    # User prompt 구성
    key_points_text = "\n".join([f"- {point}" for point in key_points])
    user_message = f"""Write a {purpose} for {audience}.

Key points to include:
{key_points_text}

Please write a professional document that covers all key points clearly and concisely."""
    
    messages = [
        {"role": "system", "content": prompt_v1['system']},
        {"role": "user", "content": user_message}
    ]
    
    # OpenAI API 호출 with Langfuse tracing (자동)
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0.7,
        max_tokens=800,
        messages=messages,
        # Langfuse metadata
        name=trace_name,
        metadata={
            "purpose": purpose,
            "audience": audience,
            "key_points_count": len(key_points),
            "version": "v1.0.0"
        }
    )
    
    return response.choices[0].message.content


def generate_email_v2(
    purpose: str,
    audience: str,
    key_points: List[str],
    style: str = "professional",
    tone: str = "neutral",
    must_include: List[str] = None,
    max_length: int = 500,
    trace_name: str = "email-generation-v2"
) -> str:
    """V2 프롬프트로 이메일 생성 (Langfuse Tracing)"""
    
    if must_include is None:
        must_include = []
    
    # User prompt 구성 (V2 템플릿)
    key_points_text = "\n".join([f"- {point}" for point in key_points])
    must_include_text = "\n".join([f"✓ {item}" for item in must_include])
    
    user_message = f"""**Document Type:** {purpose}
**Target Audience:** {audience}

**Key Points to Address:**
{key_points_text}

**Writing Requirements:**
- Style: {style}
- Tone: {tone}
- Maximum Length: {max_length} words

**Mandatory Elements (MUST INCLUDE ALL):**
{must_include_text}

Please generate a professional document that:
1. Addresses all key points comprehensively
2. Includes ALL mandatory elements listed above
3. Maintains the specified style and tone consistently
4. Stays within the word limit
5. Follows proper business document structure

Generate the document now:"""
    
    messages = [
        {"role": "system", "content": prompt_v2['system']},
        {"role": "user", "content": user_message}
    ]
    
    # OpenAI API 호출 with Langfuse tracing (자동)
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0.7,
        max_tokens=1000,
        messages=messages,
        # Langfuse metadata
        name=trace_name,
        metadata={
            "purpose": purpose,
            "audience": audience,
            "key_points_count": len(key_points),
            "style": style,
            "tone": tone,
            "must_include_count": len(must_include),
            "max_length": max_length,
            "version": "v2.0.0"
        }
    )
    
    return response.choices[0].message.content

print("✅ Email 생성 함수 정의 완료 (Tracing 활성화)")

✅ Email 생성 함수 정의 완료 (Tracing 활성화)


## 🧪 V1 & V2 테스트 실행 (최소 5건씩 Trace 남기기)

In [13]:
# V1 테스트: 처음 5개 샘플로 실행
print("🔍 V1 Prompt 테스트 (5건)\n" + "="*60)

v1_results = []
for i, sample in enumerate(dataset[:5], 1):
    print(f"\n[{i}/5] {sample['input']['purpose']}...")
    
    output = generate_email_v1(
        purpose=sample['input']['purpose'],
        audience=sample['input']['audience'],
        key_points=sample['input']['key_points'],
        trace_name=f"v1-test-{sample['id']}"
    )
    
    v1_results.append({
        "id": sample['id'],
        "output": output,
        "reference": sample['reference']
    })
    
    print(f"  ✅ 생성 완료 ({len(output)} chars)")
    time.sleep(0.5)  # Rate limit 방지

langfuse.flush()
print("\n" + "="*60)
print("✅ V1 테스트 완료! Langfuse에서 Trace 확인 가능")

🔍 V1 Prompt 테스트 (5건)

[1/5] 출장 보고서...
  ✅ 생성 완료 (1170 chars)
  ✅ 생성 완료 (1170 chars)

[2/5] 프로젝트 제안서...

[2/5] 프로젝트 제안서...
  ✅ 생성 완료 (1196 chars)
  ✅ 생성 완료 (1196 chars)

[3/5] 고객 불만 응대 이메일...

[3/5] 고객 불만 응대 이메일...
  ✅ 생성 완료 (650 chars)
  ✅ 생성 완료 (650 chars)

[4/5] 팀원 업무 지시...

[4/5] 팀원 업무 지시...
  ✅ 생성 완료 (427 chars)
  ✅ 생성 완료 (427 chars)

[5/5] 협업 제안 이메일...

[5/5] 협업 제안 이메일...
  ✅ 생성 완료 (440 chars)
  ✅ 생성 완료 (440 chars)

✅ V1 테스트 완료! Langfuse에서 Trace 확인 가능

✅ V1 테스트 완료! Langfuse에서 Trace 확인 가능


In [14]:
# V2 테스트: 동일한 5개 샘플로 실행 (비교를 위해)
print("🔍 V2 Prompt 테스트 (5건)\n" + "="*60)

v2_results = []
for i, sample in enumerate(dataset[:5], 1):
    print(f"\n[{i}/5] {sample['input']['purpose']}...")
    
    output = generate_email_v2(
        purpose=sample['input']['purpose'],
        audience=sample['input']['audience'],
        key_points=sample['input']['key_points'],
        style=sample['reference'].get('style', 'professional'),
        tone=sample['reference'].get('tone', 'neutral'),
        must_include=sample['reference'].get('must_include', []),
        max_length=sample['reference'].get('max_length', 500),
        trace_name=f"v2-test-{sample['id']}"
    )
    
    v2_results.append({
        "id": sample['id'],
        "output": output,
        "reference": sample['reference']
    })
    
    print(f"  ✅ 생성 완료 ({len(output)} chars)")
    time.sleep(0.5)

langfuse.flush()
print("\n" + "="*60)
print("✅ V2 테스트 완료! Langfuse에서 Trace 확인 가능")

🔍 V2 Prompt 테스트 (5건)

[1/5] 출장 보고서...
  ✅ 생성 완료 (1221 chars)
  ✅ 생성 완료 (1221 chars)

[2/5] 프로젝트 제안서...

[2/5] 프로젝트 제안서...
  ✅ 생성 완료 (1478 chars)
  ✅ 생성 완료 (1478 chars)

[3/5] 고객 불만 응대 이메일...

[3/5] 고객 불만 응대 이메일...
  ✅ 생성 완료 (771 chars)
  ✅ 생성 완료 (771 chars)

[4/5] 팀원 업무 지시...

[4/5] 팀원 업무 지시...
  ✅ 생성 완료 (609 chars)
  ✅ 생성 완료 (609 chars)

[5/5] 협업 제안 이메일...

[5/5] 협업 제안 이메일...
  ✅ 생성 완료 (841 chars)
  ✅ 생성 완료 (841 chars)

✅ V2 테스트 완료! Langfuse에서 Trace 확인 가능

✅ V2 테스트 완료! Langfuse에서 Trace 확인 가능


## 📤 Langfuse Dataset에 업로드

In [17]:
# Langfuse Dataset 생성/업데이트
DATASET_NAME = "email-writer-eval"

print(f"📤 Langfuse Dataset '{DATASET_NAME}' 생성 및 업로드 중...\n")

# Step 1: Dataset 먼저 생성
try:
    langfuse.create_dataset(
        name=DATASET_NAME,
        description="Email & Document Writer 평가용 데이터셋 - 15개 샘플",
        metadata={
            "version": "1.0",
            "created_date": datetime.now().isoformat(),
            "total_samples": len(dataset)
        }
    )
    print(f"✅ Dataset '{DATASET_NAME}' 생성 완료\n")
except Exception as e:
    print(f"⚠️  Dataset 이미 존재하거나 오류: {e}\n")

# Step 2: Dataset에 아이템 추가
print("📝 Dataset 아이템 추가 중...")
for i, sample in enumerate(dataset, 1):
    langfuse.create_dataset_item(
        dataset_name=DATASET_NAME,
        input=sample['input'],
        expected_output=sample['reference'],
        metadata={
            "id": sample['id'],
            "purpose": sample['input']['purpose']
        }
    )
    if i % 5 == 0:
        print(f"  진행률: {i}/{len(dataset)}")

langfuse.flush()
print(f"\n✅ Dataset 업로드 완료: {len(dataset)}개 아이템")
print(f"   Langfuse UI에서 '{DATASET_NAME}' 확인 가능")

📤 Langfuse Dataset 'email-writer-eval' 생성 및 업로드 중...

✅ Dataset 'email-writer-eval' 생성 완료

📝 Dataset 아이템 추가 중...
✅ Dataset 'email-writer-eval' 생성 완료

📝 Dataset 아이템 추가 중...
  진행률: 5/15
  진행률: 5/15
  진행률: 10/15
  진행률: 10/15
  진행률: 15/15

✅ Dataset 업로드 완료: 15개 아이템
   Langfuse UI에서 'email-writer-eval' 확인 가능
  진행률: 15/15

✅ Dataset 업로드 완료: 15개 아이템
   Langfuse UI에서 'email-writer-eval' 확인 가능


## 📊 Evaluation 함수 정의

In [18]:
def evaluate_mandatory_items(output: str, must_include: List[str]) -> Dict:
    """필수 항목 포함 여부 체크 (정량)"""
    found_count = sum(1 for item in must_include if item.lower() in output.lower())
    total = len(must_include)
    
    return {
        "mandatory_items_found": found_count,
        "mandatory_items_total": total,
        "mandatory_items_rate": found_count / total if total > 0 else 1.0
    }


def evaluate_length(output: str, max_length: int) -> Dict:
    """길이 제한 준수 체크 (정량)"""
    word_count = len(output.split())
    compliant = word_count <= max_length
    
    return {
        "word_count": word_count,
        "max_length": max_length,
        "length_compliant": compliant,
        "length_ratio": word_count / max_length if max_length > 0 else 0
    }


def llm_judge_tone_consistency(output: str, expected_tone: str) -> int:
    """LLM-as-Judge: 톤 일치도 평가 (0-5점)"""
    judge_prompt = f"""다음 문서의 톤(tone)이 '{expected_tone}' 톤과 얼마나 일치하는지 0-5점으로 평가하세요.

평가 기준:
- 5점: 완벽하게 일치
- 4점: 대부분 일치하나 일부 불일치
- 3점: 중간 정도 일치
- 2점: 많이 불일치
- 1점: 거의 불일치
- 0점: 완전 불일치

문서:
{output}

숫자만 답변하세요 (0-5):"""
    
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0.3,
        messages=[{"role": "user", "content": judge_prompt}]
    )
    
    score_text = response.choices[0].message.content.strip()
    try:
        score = int(score_text)
        return min(max(score, 0), 5)  # Clamp to 0-5
    except:
        return 3  # Default to middle score if parsing fails


def llm_judge_edit_need(output: str) -> int:
    """LLM-as-Judge: 편집 필요도 평가 (0-5점, 낮을수록 좋음)"""
    judge_prompt = f"""다음 비즈니스 문서를 바로 실무에서 사용하기 위해 얼마나 많은 편집이 필요한지 0-5점으로 평가하세요.

평가 기준:
- 0점: 편집 불필요, 바로 사용 가능
- 1점: 극소량의 편집만 필요
- 2점: 일부 편집 필요
- 3점: 상당한 편집 필요
- 4점: 대부분 다시 작성 필요
- 5점: 완전히 다시 작성 필요

문서:
{output}

숫자만 답변하세요 (0-5):"""
    
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0.3,
        messages=[{"role": "user", "content": judge_prompt}]
    )
    
    score_text = response.choices[0].message.content.strip()
    try:
        score = int(score_text)
        return min(max(score, 0), 5)
    except:
        return 3


def evaluate_output(output: str, reference: Dict) -> Dict:
    """전체 평가 실행"""
    results = {}
    
    # 정량 평가
    results.update(evaluate_mandatory_items(output, reference.get('must_include', [])))
    results.update(evaluate_length(output, reference.get('max_length', 500)))
    
    # 정성 평가 (LLM-as-Judge)
    results['tone_consistency_score'] = llm_judge_tone_consistency(
        output, reference.get('tone', 'neutral')
    )
    results['edit_need_score'] = llm_judge_edit_need(output)
    
    return results

print("✅ Evaluation 함수 정의 완료")

✅ Evaluation 함수 정의 완료


## 📈 V1 vs V2 평가 실행

In [19]:
print("📊 V1 결과 평가 중...\n" + "="*60)

v1_eval_results = []
for i, result in enumerate(v1_results, 1):
    print(f"[{i}/{len(v1_results)}] Evaluating {result['id']}...")
    eval_result = evaluate_output(result['output'], result['reference'])
    v1_eval_results.append(eval_result)
    time.sleep(0.3)

print("\n" + "="*60)
print("✅ V1 평가 완료\n")

# V1 평균 계산
v1_avg = {
    "mandatory_items_rate": sum(r['mandatory_items_rate'] for r in v1_eval_results) / len(v1_eval_results),
    "length_compliant_rate": sum(r['length_compliant'] for r in v1_eval_results) / len(v1_eval_results),
    "avg_tone_score": sum(r['tone_consistency_score'] for r in v1_eval_results) / len(v1_eval_results),
    "avg_edit_need": sum(r['edit_need_score'] for r in v1_eval_results) / len(v1_eval_results),
}

print("V1 평균 성능:")
print(f"  필수 항목 포함률: {v1_avg['mandatory_items_rate']*100:.1f}%")
print(f"  길이 제한 준수율: {v1_avg['length_compliant_rate']*100:.1f}%")
print(f"  톤 일치도: {v1_avg['avg_tone_score']:.2f}/5")
print(f"  편집 필요도: {v1_avg['avg_edit_need']:.2f}/5 (낮을수록 좋음)")

📊 V1 결과 평가 중...
[1/5] Evaluating e001...
[2/5] Evaluating e002...
[2/5] Evaluating e002...
[3/5] Evaluating e003...
[3/5] Evaluating e003...
[4/5] Evaluating e004...
[4/5] Evaluating e004...
[5/5] Evaluating e005...
[5/5] Evaluating e005...

✅ V1 평가 완료

V1 평균 성능:
  필수 항목 포함률: 0.0%
  길이 제한 준수율: 100.0%
  톤 일치도: 4.80/5
  편집 필요도: 1.20/5 (낮을수록 좋음)

✅ V1 평가 완료

V1 평균 성능:
  필수 항목 포함률: 0.0%
  길이 제한 준수율: 100.0%
  톤 일치도: 4.80/5
  편집 필요도: 1.20/5 (낮을수록 좋음)


In [20]:
print("📊 V2 결과 평가 중...\n" + "="*60)

v2_eval_results = []
for i, result in enumerate(v2_results, 1):
    print(f"[{i}/{len(v2_results)}] Evaluating {result['id']}...")
    eval_result = evaluate_output(result['output'], result['reference'])
    v2_eval_results.append(eval_result)
    time.sleep(0.3)

print("\n" + "="*60)
print("✅ V2 평가 완료\n")

# V2 평균 계산
v2_avg = {
    "mandatory_items_rate": sum(r['mandatory_items_rate'] for r in v2_eval_results) / len(v2_eval_results),
    "length_compliant_rate": sum(r['length_compliant'] for r in v2_eval_results) / len(v2_eval_results),
    "avg_tone_score": sum(r['tone_consistency_score'] for r in v2_eval_results) / len(v2_eval_results),
    "avg_edit_need": sum(r['edit_need_score'] for r in v2_eval_results) / len(v2_eval_results),
}

print("V2 평균 성능:")
print(f"  필수 항목 포함률: {v2_avg['mandatory_items_rate']*100:.1f}%")
print(f"  길이 제한 준수율: {v2_avg['length_compliant_rate']*100:.1f}%")
print(f"  톤 일치도: {v2_avg['avg_tone_score']:.2f}/5")
print(f"  편집 필요도: {v2_avg['avg_edit_need']:.2f}/5 (낮을수록 좋음)")

📊 V2 결과 평가 중...
[1/5] Evaluating e001...
[2/5] Evaluating e002...
[2/5] Evaluating e002...
[3/5] Evaluating e003...
[3/5] Evaluating e003...
[4/5] Evaluating e004...
[4/5] Evaluating e004...
[5/5] Evaluating e005...
[5/5] Evaluating e005...

✅ V2 평가 완료

V2 평균 성능:
  필수 항목 포함률: 53.3%
  길이 제한 준수율: 100.0%
  톤 일치도: 4.80/5
  편집 필요도: 1.20/5 (낮을수록 좋음)

✅ V2 평가 완료

V2 평균 성능:
  필수 항목 포함률: 53.3%
  길이 제한 준수율: 100.0%
  톤 일치도: 4.80/5
  편집 필요도: 1.20/5 (낮을수록 좋음)


## 📋 V1 vs V2 비교 리포트

In [21]:
import pandas as pd

# 비교표 생성
comparison_df = pd.DataFrame({
    "지표": [
        "필수 항목 포함률 (%)",
        "길이 제한 준수율 (%)",
        "톤 일치도 (0-5)",
        "편집 필요도 (0-5, 낮을수록 좋음)"
    ],
    "V1 (dev)": [
        f"{v1_avg['mandatory_items_rate']*100:.1f}%",
        f"{v1_avg['length_compliant_rate']*100:.1f}%",
        f"{v1_avg['avg_tone_score']:.2f}",
        f"{v1_avg['avg_edit_need']:.2f}"
    ],
    "V2 (production)": [
        f"{v2_avg['mandatory_items_rate']*100:.1f}%",
        f"{v2_avg['length_compliant_rate']*100:.1f}%",
        f"{v2_avg['avg_tone_score']:.2f}",
        f"{v2_avg['avg_edit_need']:.2f}"
    ],
    "개선도": [
        f"{(v2_avg['mandatory_items_rate'] - v1_avg['mandatory_items_rate'])*100:+.1f}%p",
        f"{(v2_avg['length_compliant_rate'] - v1_avg['length_compliant_rate'])*100:+.1f}%p",
        f"{v2_avg['avg_tone_score'] - v1_avg['avg_tone_score']:+.2f}",
        f"{v2_avg['avg_edit_need'] - v1_avg['avg_edit_need']:+.2f}"
    ]
})

print("\n" + "="*80)
print("📊 V1 vs V2 성능 비교표")
print("="*80)
print(comparison_df.to_string(index=False))
print("="*80)

# CSV로 저장
comparison_df.to_csv("v1_vs_v2_comparison.csv", index=False, encoding='utf-8-sig')
print("\n✅ 비교표 저장: v1_vs_v2_comparison.csv")


📊 V1 vs V2 성능 비교표
                   지표 V1 (dev) V2 (production)     개선도
        필수 항목 포함률 (%)     0.0%           53.3% +53.3%p
        길이 제한 준수율 (%)   100.0%          100.0%  +0.0%p
          톤 일치도 (0-5)     4.80            4.80   +0.00
편집 필요도 (0-5, 낮을수록 좋음)     1.20            1.20   +0.00

✅ 비교표 저장: v1_vs_v2_comparison.csv


## 🎯 주요 발견사항 및 개선안

In [22]:
# 실패 사례 분석 (V1에서 점수가 낮았던 케이스)
print("🔍 V1 주요 실패 사례 (Top 3)\n" + "="*60)

# Edit need가 가장 높은 3개 찾기
v1_with_scores = [(i, r) for i, r in enumerate(v1_eval_results)]
v1_worst = sorted(v1_with_scores, key=lambda x: x[1]['edit_need_score'], reverse=True)[:3]

for rank, (idx, eval_result) in enumerate(v1_worst, 1):
    print(f"\n{rank}. ID: {v1_results[idx]['id']}")
    print(f"   편집 필요도: {eval_result['edit_need_score']}/5")
    print(f"   필수 항목 포함률: {eval_result['mandatory_items_rate']*100:.0f}%")
    print(f"   문제점: 구조화 부족, 필수 요소 누락, 톤 불일치")

print("\n" + "="*60)
print("\n💡 다음 개선안 (V3 고려사항)")
print("="*60)
print("1. Few-shot examples 추가하여 형식 일관성 향상")
print("2. JSON Schema로 출력 구조화 강제")
print("3. Chain-of-Thought로 필수 항목 체크리스트 먼저 작성")
print("4. 업종별/목적별 프롬프트 세분화 (이메일 vs 보고서 vs 제안서)")
print("5. Self-critique 단계 추가 (생성 후 자기 검증)")
print("="*60)

🔍 V1 주요 실패 사례 (Top 3)

1. ID: e003
   편집 필요도: 2/5
   필수 항목 포함률: 0%
   문제점: 구조화 부족, 필수 요소 누락, 톤 불일치

2. ID: e001
   편집 필요도: 1/5
   필수 항목 포함률: 0%
   문제점: 구조화 부족, 필수 요소 누락, 톤 불일치

3. ID: e002
   편집 필요도: 1/5
   필수 항목 포함률: 0%
   문제점: 구조화 부족, 필수 요소 누락, 톤 불일치


💡 다음 개선안 (V3 고려사항)
1. Few-shot examples 추가하여 형식 일관성 향상
2. JSON Schema로 출력 구조화 강제
3. Chain-of-Thought로 필수 항목 체크리스트 먼저 작성
4. 업종별/목적별 프롬프트 세분화 (이메일 vs 보고서 vs 제안서)
5. Self-critique 단계 추가 (생성 후 자기 검증)
